# Introduction to Q-Learning


In [ ]:
# Introduction to Q-Learning

# Markdown cell explaining the Q-Learning algorithm
"""
### What is Q-Learning?

Q-Learning is a model-free reinforcement learning algorithm used to find the optimal action-selection policy for a given finite Markov Decision Process (MDP). It is based on the concept of learning the Q-value, which represents the expected utility of taking a certain action in a given state, and following the optimal policy thereafter.

The Q-value is updated iteratively using the following update rule:

\[
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left( R_t + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t) \right)
\]

Where:
- \( Q(s_t, a_t) \): Current Q-value for state \( s_t \) and action \( a_t \).
- \( \alpha \): Learning rate, which determines how much new information overrides the old information.
- \( R_t \): Reward received after taking action \( a_t \) in state \( s_t \).
- \( \gamma \): Discount factor, which determines the importance of future rewards.
- \( \max_{a} Q(s_{t+1}, a) \): Maximum Q-value for the next state \( s_{t+1} \) over all possible actions.

The goal of Q-Learning is to iteratively update the Q-values until they converge to the optimal Q-values, which can then be used to derive the optimal policy.
"""

# Import Required Libraries


### Importing Libraries

In this section, we import the necessary libraries for implementing the Q-Learning algorithm:

- **NumPy**: A powerful library for numerical computations. It is used for handling arrays, performing mathematical operations, and managing the Q-value table efficiently.
- **Matplotlib**: A library for creating visualizations. It will be used to plot the results and visualize the learning process.

These libraries are essential for implementing and analyzing the Q-Learning algorithm effectively.


In [ ]:
# Import Required Libraries

import numpy as np
import matplotlib.pyplot as plt

# Define the Grid Graph
Explain the grid graph structure, where each state is a node, and edges represent possible actions with directions.

In [ ]:
# Define the Grid Graph

"""
### Defining the Grid Graph

The grid graph represents the environment in which the agent operates. Each state in the grid is represented as a node, and the edges between nodes represent possible actions (e.g., moving up, down, left, or right). The graph is defined as a dictionary where:
- Keys are the states (nodes) represented as strings.
- Values are lists of tuples, where each tuple contains:
  - The neighboring state (node) the agent can move to.
  - The action (direction) required to move to that neighboring state.

This structure allows us to model the environment and define the possible transitions between states.
"""

grid_graph = {
    "1": [("2", "right"), ("6", "down")],
    "2": [("1", "left"), ("3", "right"), ("7", "down")],
    "3": [("2", "left"), ("4", "right"), ("8", "down")],
    "4": [("3", "left"), ("5", "right"), ("9", "down")],
    "5": [("4", "left"), ("10", "down")],

    "6": [("1", "up"), ("7", "right"), ("11", "down")],
    "7": [("2", "up"), ("6", "left"), ("8", "right"), ("12", "down")],
    "8": [("3", "up"), ("7", "left"), ("9", "right"), ("13", "down")],
    "9": [("4", "up"), ("8", "left"), ("10", "right"), ("14", "down")],
    "10": [("5", "up"), ("9", "left"), ("15", "down")],

    "11": [("6", "up"), ("12", "right"), ("16", "down")],
    "12": [("7", "up"), ("11", "left"), ("13", "right"), ("17", "down")],
    "13": [("8", "up"), ("12", "left"), ("14", "right"), ("18", "down")],
    "14": [("9", "up"), ("13", "left"), ("15", "right"), ("19", "down")],
    "15": [("10", "up"), ("14", "left"), ("20", "down")],

    "16": [("11", "up"), ("17", "right"), ("21", "down")],
    "17": [("12", "up"), ("16", "left"), ("18", "right"), ("22", "down")],
    "18": [("13", "up"), ("17", "left"), ("19", "right"), ("23", "down")],
    "19": [("14", "up"), ("18", "left"), ("20", "right"), ("24", "down")],
    "20": [("15", "up"), ("19", "left"), ("25", "down")],

    "21": [("16", "up"), ("22", "right")],
    "22": [("17", "up"), ("21", "left"), ("23", "right")],
    "23": [("18", "up"), ("22", "left"), ("24", "right")],
    "24": [("19", "up"), ("23", "left"), ("25", "right")],
    "25": [("20", "up"), ("24", "left")]
}

# Initialize Q-States and Rewards
Add a markdown cell explaining the initialization of Q-values for each state-action pair and the reward structure, including penalties and goal rewards.

In [ ]:
# Initialize Q-States and Rewards

"""
### Initializing Q-Values and Rewards

In this section, we initialize the Q-values and rewards for each state in the grid environment:

- **Q-Values**: Represent the expected utility of taking a specific action in a given state. Initially, all Q-values are set to 0 for every state-action pair.
- **Rewards**: Define the immediate reward received upon entering a state. Most states have a default reward of -1 to encourage the agent to find the shortest path to the goal. Special states include:
  - **State 0**: A penalty state with a reward of -10.
  - **State 25**: The goal state with a reward of +10.

This setup ensures that the agent is incentivized to reach the goal state while avoiding unnecessary steps or penalties.
"""

# Initialize Q-values for each state-action pair
Q_states = {str(i): {"right": 0, "up": 0, "left": 0, "down": 0} for i in range(1, 26)}

# Define rewards for each state
rewards = {str(i): -1 for i in range(1, 26)}
rewards["0"] = -10  # Penalty state
rewards["25"] = 10  # Goal state

# Helper Functions
Explain the purpose of helper functions for retrieving neighbor states, getting and setting Q-values, and fetching rewards.

In [ ]:
# Helper Functions

"""
### Helper Functions

This section defines helper functions that are essential for the Q-Learning algorithm. These functions include:

1. **`get_neighbor_states(state)`**: Retrieves the neighboring states and possible actions for a given state.
2. **`get_state_Q(state, action)`**: Fetches the Q-value for a specific state-action pair.
3. **`set_state_Q(state, action, new_value)`**: Updates the Q-value for a specific state-action pair.
4. **`get_reward(state)`**: Returns the reward associated with a given state.

These functions abstract the operations on the grid graph, Q-values, and rewards, making the implementation modular and easier to understand.
"""

# Function to retrieve neighboring states and actions
def get_neighbor_states(state: str) -> list[tuple[str, str]]:
    """
    Retrieves the neighboring states and possible actions for a given state.

    Args:
        state (str): The current state.

    Returns:
        list[tuple[str, str]]: A list of tuples where each tuple contains:
            - The neighboring state.
            - The action required to move to that state.
    """
    return grid_graph[state]

# Function to get the Q-value for a specific state-action pair
def get_state_Q(state: str, action: str) -> int:
    """
    Fetches the Q-value for a specific state-action pair.

    Args:
        state (str): The current state.
        action (str): The action to be taken.

    Returns:
        int: The Q-value for the given state-action pair.
    """
    return Q_states[state][action]

# Function to set the Q-value for a specific state-action pair
def set_state_Q(state: str, action: str, new_value: float) -> None:
    """
    Updates the Q-value for a specific state-action pair.

    Args:
        state (str): The current state.
        action (str): The action to be taken.
        new_value (float): The new Q-value to be set.
    """
    Q_states[state][action] = new_value

# Function to get the reward for a specific state
def get_reward(state: str) -> int:
    """
    Returns the reward associated with a given state.

    Args:
        state (str): The current state.

    Returns:
        int: The reward for the given state.
    """
    return rewards[state]

# Q-Value Update Function
Add a markdown cell explaining the Q-value update function, including the role of learning rate, discount factor, and step penalty.

### Q-Value Update Function

The Q-value update function is a critical component of the Q-Learning algorithm. It updates the Q-value for a given state-action pair based on the reward received and the maximum Q-value of the next state. The update rule is as follows:

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left( R_t + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t) \right)
$$

#### Components:
- **Learning Rate ($ \alpha $)**: Determines how much the new information overrides the old information. A higher value means the agent learns faster but may overshoot the optimal value.
- **Discount Factor ($ \gamma $)**: Determines the importance of future rewards. A value close to 1 emphasizes long-term rewards, while a value close to 0 focuses on immediate rewards.
- **Step Penalty**: An additional penalty applied to discourage unnecessary steps, encouraging the agent to find the shortest path to the goal.

This function ensures that the agent learns the optimal policy by iteratively updating the Q-values based on the observed rewards and transitions.


In [ ]:
# Q-Value Update Function


def get_Q_update(
    state_Q: float,
    state_reward: float,
    neighbor_states: list[str],
    learning_rate: float = 0.5,
    discount_factor: float = 0.5,
    step_penalty: float = 0
) -> float:
    """
    Updates the Q-value for a given state-action pair.

    Args:
        state_Q (float): Current Q-value of the state-action pair.
        state_reward (float): Reward received for the current state.
        neighbor_states (list[str]): List of neighboring states and actions.
        learning_rate (float, optional): Learning rate (alpha). Defaults to 0.5.
        discount_factor (float, optional): Discount factor (gamma). Defaults to 0.5.
        step_penalty (float, optional): Penalty for taking a step. Defaults to 0.

    Returns:
        float: Updated Q-value for the state-action pair.
    """
    # Calculate the maximum Q-value among the neighboring states
    neighbor_states_Q = map(lambda x: get_state_Q(*x), neighbor_states)
    
    # Apply the Q-value update formula
    return state_Q + learning_rate * (
        (state_reward + step_penalty) + discount_factor * max(neighbor_states_Q) - state_Q
    )

# Neighbor Q-Values and Next State Selection
Explain the logic for calculating neighbor Q-values and selecting the next state based on the highest Q-value.

### Calculating Neighbor Q-Values

In this section, we calculate the Q-values of all neighboring states for a given current state. This helps the agent evaluate the potential outcomes of each possible action. The function `get_neighbor_states_Q` retrieves the Q-values for all neighboring states and their corresponding actions.

#### Formula:
For each neighboring state \( s' \) and action \( a \), the Q-value is given by:
\[
Q(s, a) = Q_{\text{states}}[s][a]
\]

This step is crucial for determining the best action to take from the current state.

In [ ]:
# Neighbor Q-Values 

def get_neighbor_states_Q(current_state: str, neighbor_states: list[tuple[str, str]]) -> list[float]:
    """
    Retrieves the Q-values for all neighboring states and actions.

    Args:
        current_state (str): The current state.
        neighbor_states (list[tuple[str, str]]): List of neighboring states and actions.

    Returns:
        list[float]: List of Q-values for the neighboring states and actions.
    """
    return list(map(lambda x: get_state_Q(current_state, x[1]), neighbor_states))



### Selecting the Next State

After calculating the Q-values of all neighboring states, the agent selects the next state based on the action with the highest Q-value. The function `get_next_state` identifies the best action and its corresponding state.

#### Logic:
1. Compute the Q-values for all neighboring states.
2. Identify the action with the maximum Q-value.
3. Return the corresponding neighboring state and action.

This ensures that the agent follows a greedy policy to maximize its expected reward.

In [ ]:
# Next State Selection

def get_next_state(current_state: str, neighbor_states: list[tuple[str, str]]) -> tuple[str, str]:
    """
    Selects the next state based on the highest Q-value among neighboring states.

    Args:
        current_state (str): The current state.
        neighbor_states (list[tuple[str, str]]): List of neighboring states and actions.

    Returns:
        tuple[str, str]: The next state and the action leading to it.
    """
    neighbor_states_Q = get_neighbor_states_Q(current_state, neighbor_states)
    max_index = np.argmax(neighbor_states_Q)
    return neighbor_states[max_index]

# Main Q-Learning Loop


### Main Q-Learning Loop

This section implements the main loop of the Q-Learning algorithm. The loop iteratively updates the Q-values, selects actions, and transitions between states until the goal state is reached or a maximum number of iterations is exceeded.

#### Steps:
1. **Initialization**: Start from an initial state (e.g., state "1").
2. **Neighbor States**: Retrieve the neighboring states and their Q-values.
3. **Action Selection**: Choose the next action based on the highest Q-value (greedy policy).
4. **Q-Value Update**: Update the Q-value of the current state-action pair using the Q-Learning update rule.
5. **Transition**: Move to the next state and repeat the process.
6. **Stopping Condition**: Stop when the goal state is reached or the maximum number of iterations is reached.

This loop ensures that the agent learns the optimal policy by exploring the environment and updating its Q-values based on observed rewards and transitions.


In [ ]:
# Main Q-Learning Loop

current_state = "1"  # Initialize the starting state
states_follow_up = [current_state]  # Track the sequence of states visited

# Maximum number of iterations to prevent infinite loops
max_iterations = 1_000_000

for i in range(max_iterations):
    print(f"Step {i + 1}:")
    print(f"\tCurrent state: {current_state}")

    # Check if the goal state is reached
    if current_state == "25":
        print(f"Goal state found at iteration {i + 1}")
        break

    # Retrieve neighboring states and their Q-values
    neighbor_states = get_neighbor_states(state=current_state)
    neighbor_states_Q = get_neighbor_states_Q(current_state=current_state, neighbor_states=neighbor_states)
    print(f"\tNeighbor states: {neighbor_states}")
    print(f"\tNeighbor states Q-values: {neighbor_states_Q}")

    # Select the next state based on the highest Q-value
    next_state = get_next_state(current_state=current_state, neighbor_states=neighbor_states)
    next_state_Q = get_state_Q(state=current_state, action=next_state[1])
    next_state_reward = get_reward(next_state[0])
    print(f'\tNext action: "{next_state[1]}" to state {next_state[0]} with Q-value = {next_state_Q} and reward = {next_state_reward}')

    # Update the Q-value for the current state-action pair
    updated_Q = get_Q_update(
        state_Q=next_state_Q,
        state_reward=next_state_reward,
        neighbor_states=neighbor_states,
        learning_rate=0.3,
        discount_factor=0.9,
        step_penalty=-0.5
    )
    set_state_Q(state=current_state, action=next_state[1], new_value=updated_Q)
    print(f"\tUpdated Q-value: {get_state_Q(state=current_state, action=next_state[1])}")

    # Transition to the next state
    current_state = next_state[0]
    states_follow_up.append(current_state)